<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

Our first example is minimalistic - find the largest eigenvalue of
a diagonal matrix by solving the non-generalized eigenvalue problem
``A x = lambda x`` without constraints or preconditioning.


In [ ]:
import numpy as np
from scipy.sparse import spdiags
from scipy.sparse.linalg import LinearOperator, aslinearoperator
from scipy.sparse.linalg import lobpcg

The square matrix size is


In [ ]:
n = 100

and its diagonal entries are 1, ..., 100 defined by


In [ ]:
vals = np.arange(1, n + 1).astype(np.int16)

The first mandatory input parameter in this test is
the sparse diagonal matrix `A`
of the eigenvalue problem ``A x = lambda x`` to solve.


In [ ]:
A = spdiags(vals, 0, n, n)
A = A.astype(np.int16)
A.toarray()

array([[  1,   0,   0, ...,   0,   0,   0],
       [  0,   2,   0, ...,   0,   0,   0],
       [  0,   0,   3, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,  98,   0,   0],
       [  0,   0,   0, ...,   0,  99,   0],
       [  0,   0,   0, ...,   0,   0, 100]], dtype=int16)

The second mandatory input parameter `X` is a 2D array with the
row dimension determining the number of requested eigenvalues.
`X` is an initial guess for targeted eigenvectors.
`X` must have linearly independent columns.
If no initial approximations available, randomly oriented vectors
commonly work best, e.g., with components normally distributed
around zero or uniformly distributed on the interval [-1 1].
Setting the initial approximations to dtype ``np.float32``
forces all iterative values to dtype ``np.float32`` speeding up
the run while still allowing accurate eigenvalue computations.


In [ ]:
k = 1
rng = np.random.default_rng()
X = rng.normal(size=(n, k))
X = X.astype(np.float32)

In [ ]:
eigenvalues, _ = lobpcg(A, X, maxiter=60)
eigenvalues

array([100.])

In [ ]:
eigenvalues.dtype

dtype('float32')

`lobpcg` needs only access the matrix product with `A` rather
then the matrix itself. Since the matrix `A` is diagonal in
this example, one can write a function of the matrix product
``A @ X`` using the diagonal values ``vals`` only, e.g., by
element-wise multiplication with broadcasting in the lambda-function


In [ ]:
A_lambda = lambda X: vals[:, np.newaxis] * X

or the regular function


In [ ]:
def A_matmat(X):
    return vals[:, np.newaxis] * X

and use the handle to one of these callables as an input


In [ ]:
eigenvalues, _ = lobpcg(A_lambda, X, maxiter=60)
eigenvalues

array([100.])

In [ ]:
eigenvalues, _ = lobpcg(A_matmat, X, maxiter=60)
eigenvalues

array([100.])

The traditional callable `LinearOperator` is no longer
necessary but still supported as the input to `lobpcg`.
Specifying ``matmat=A_matmat`` explicitly improves performance. 


In [ ]:
A_lo = LinearOperator((n, n), matvec=A_matmat, matmat=A_matmat, dtype=np.int16)
eigenvalues, _ = lobpcg(A_lo, X, maxiter=80)
eigenvalues

array([100.])

The least efficient callable option is `aslinearoperator`:


In [ ]:
eigenvalues, _ = lobpcg(aslinearoperator(A), X, maxiter=80)
eigenvalues

array([100.])

We now switch to computing the three smallest eigenvalues specifying


In [ ]:
k = 3
X = np.random.default_rng().normal(size=(n, k))

and ``largest=False`` parameter


In [ ]:
eigenvalues, _ = lobpcg(A, X, largest=False, maxiter=90)
print(eigenvalues)  

[1. 2. 3.]

The next example illustrates computing 3 smallest eigenvalues of
the same matrix `A` given by the function handle ``A_matmat`` but
with constraints and preconditioning.

Constraints - an optional input parameter is a 2D array comprising
of column vectors that the eigenvectors must be orthogonal to


In [ ]:
Y = np.eye(n, 3)

The preconditioner acts as the inverse of `A` in this example, but
in the reduced precision ``np.float32`` even though the initial `X`
and thus all iterates and the output are in full ``np.float64``.


In [ ]:
inv_vals = 1./vals
inv_vals = inv_vals.astype(np.float32)
M = lambda X: inv_vals[:, np.newaxis] * X

Let us now solve the eigenvalue problem for the matrix `A` first
without preconditioning requesting 80 iterations


In [ ]:
eigenvalues, _ = lobpcg(A_matmat, X, Y=Y, largest=False, maxiter=80)
eigenvalues

array([4., 5., 6.])

In [ ]:
eigenvalues.dtype

dtype('float64')

With preconditioning we need only 20 iterations from the same `X`


In [ ]:
eigenvalues, _ = lobpcg(A_matmat, X, Y=Y, M=M, largest=False, maxiter=20)
eigenvalues

array([4., 5., 6.])

Note that the vectors passed in `Y` are the eigenvectors of the 3
smallest eigenvalues. The results returned above are orthogonal to those.

The primary matrix `A` may be indefinite, e.g., after shifting
``vals`` by 50 from 1, ..., 100 to -49, ..., 50, we still can compute
the 3 smallest or largest eigenvalues.


In [ ]:
vals = vals - 50
X = rng.normal(size=(n, k))
eigenvalues, _ = lobpcg(A_matmat, X, largest=False, maxiter=99)
eigenvalues

array([-49., -48., -47.])

In [ ]:
eigenvalues, _ = lobpcg(A_matmat, X, largest=True, maxiter=99)
eigenvalues

array([50., 49., 48.])